In [ ]:
import numpy as np
import pandas as pd
import re
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
import pickle
from tensorflow.keras.models import model_from_json


In [ ]:
jsonpath = "Dataset for Detection of Cyber-Trolls.json"
csvpath =  "CyberBullyingTypesDataset.csv"  
pickle_model_path = "BI LSTM/models/model.pkl"
keras_model_path = "BI LSTM/models/model.keras"

In [ ]:

def clean_text(text):
    text = re.sub(r"http\S+|www\S+", "", text) 
    text = re.sub(r"@\w+", "", text)  
    text = text.lower()  
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  
    text = re.sub(r"\s+", " ", text).strip() 
    return text

In [ ]:
data = []
with open(jsonpath, 'r') as file:
    for line in file:
        try:
            data.append(json.loads(line))  
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line: {line}") 
            continue

json_texts = [item['content'] for item in data]
json_labels = [item['annotation']['label'][0] for item in data]  

df = pd.read_csv(csvpath)

csv_texts = df['Tweet'].astype(str).tolist()  
csv_labels = df['Class'].astype(str).tolist()  

combined_texts = json_texts + csv_texts
combined_labels = json_labels + csv_labels

# combined_texts = json_texts 
# combined_labels = json_labels 




texts = [clean_text(text) for text in combined_texts]



cleaned_df = pd.DataFrame({'texts': texts, 'label': combined_labels})
cleaned_df['label'] = cleaned_df['label'].str.strip().str.lower()  
cleaned_df['label'] = cleaned_df['label'].apply(lambda x: 1 if x != '0' else 0)


cleaned_df = cleaned_df.dropna(subset=['texts'])  
cleaned_df = cleaned_df[cleaned_df['texts'].str.strip() != '']  
cleaned_df = cleaned_df.drop_duplicates(subset=['texts']) 
cleaned_df.to_csv("cleaned_dataset.csv", index=False)

print("Cleaned dataset saved successfully.")

In [ ]:
import csv
import re

with open('8000other.txt', 'r', encoding='utf-8') as file:
    sentences = file.readlines()

cleaned_sentences = [clean_text(sentence) for sentence in sentences]

with open('bullying_data3.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['text', 'label'])  

    for sentence in cleaned_sentences:
        if sentence:  
            writer.writerow([sentence, 0])

print("CSV file created successfully with cleaned data.")


CSV file created successfully with cleaned data.


In [ ]:
import pandas as pd
import re
def clean_and_split_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text) 
    words = text.lower().split() 
    return words
df = pd.read_csv('cleaned_dataset.csv')
texts = df['texts'].astype(str).tolist()
all_words = []
for text in texts:
    all_words.extend(clean_and_split_text(text))
unique_words = set(all_words)
num_unique_words = len(unique_words)
print(f"Number of unique words: {num_unique_words}")

Number of unique words: 45405
